In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys

import pandas as pd

In [2]:
chrome_options = Options()

In [3]:
chrome_options.add_experimental_option("useAutomationExtension", False)

In [4]:
season = '2017'

In [5]:
blues_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=93'
brumbies_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=150'
bulls_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=97'
chiefs_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=91'
crusaders_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=96'
highlanders_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=95'
hurricanes_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=94'
jaguares_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=19650'
lions_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=63'
rebels_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=12'
waratahs_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=200'
reds_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=250'
sharks_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=98'
stormers_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=99'
sunwolves_link = 'https://super.rugby/superrugby/teams/?competition=205&season=' + season + '&team=19700'

In [6]:
team_links = [blues_link, brumbies_link, bulls_link, chiefs_link, crusaders_link, highlanders_link, hurricanes_link,
             jaguares_link, lions_link, rebels_link, waratahs_link, reds_link, sharks_link, stormers_link, sunwolves_link]

In [7]:
def grab_source_links(link):
    
    driver = webdriver.Chrome(r'C:\Users\jcoet\Projects\media_matters\chromedriver')
    
    driver.get(link)
    
    # chill
    sleep(2)
    
    # scroll page
    actions = ActionChains(driver)
    for _ in range(1):
        actions.send_keys(Keys.PAGE_DOWN).perform()
    
    # chill
    sleep(3)

    source_data = driver.page_source

    # Throw your source into BeautifulSoup and start parsing
    bs_data = bs(source_data)
    
    fixture_links = bs_data.find_all('td', class_='Opta-Outer')
    
    links = []
    
    for fixture_link in fixture_links:
        
        try:
            fixture_link = fixture_link.find('a')
            
        except TypeError:
            pass
        except AttributeError:
            pass
            
        links.append(fixture_link)
        
    match_links = []
        
    for item in links:
        try:
            match_links.append('https://super.rugby' + str(item).split('"')[1])
        except IndexError:
            pass
        
    driver.close()
    print('Done!')
            
    return match_links

In [8]:
all_links = []

for link in team_links:
    
    all_links.extend(grab_source_links(link))

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!


In [9]:
# remove duplicates - there are defnitely duplicates as the teams play against each other...

all_links = list(set(all_links))

In [10]:
# let's see what we got

all_links

['https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517067',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517102',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517166',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517032',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517121',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517063',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517057',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517015',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517043',
 'https://super.rugby/superrugby/match-centre/?season=2017&amp;competition=205&amp;match=517042',
 'https://super.rugb

In [13]:
# now to scrape the source code for the data we need from each match

rounds_lst = []
team_one_lst = []
team_two_lst = []
team_one_score_lst = []
team_two_score_lst = []
referee_lst = []
venue_lst = []
team_one_penalties_lst = []
team_two_penalties_lst = []
team_one_reds_lst = []
team_two_reds_lst = []
team_one_yellows_lst = []
team_two_yellows_lst = []

def grab_link_stats(link):
    
    driver = webdriver.Chrome(r'C:\Users\jcoet\Projects\media_matters\chromedriver')
    
    driver.get(link)
    
    # chill
    sleep(2)
    
    # scroll page
    actions = ActionChains(driver)
    for _ in range(1):
        actions.send_keys(Keys.PAGE_DOWN).perform()
    
    # chill
    sleep(3)

    source_data = driver.page_source

    # As before, throw your source into BeautifulSoup and start parsing
    bs_data = bs(source_data)
    
    try:
        rounds = bs_data.find('button', class_='btn btn-secondary btn-block dropdown-toggle').text
    except AttributeError:
        rounds = 'None'

    rounds_lst.append(rounds)
    
    try:
    
        team_one = bs_data.find('table', class_='Opta-MatchHeader Opta-MatchHeader-Crested').find_all('td')[1].text.replace('\n        ', '').replace('\n      ', '')
    except AttributeError:
        team_one = 'None'
    
    team_one_lst.append(team_one)
    
    try:
        team_one_score = bs_data.find('table', class_='Opta-MatchHeader Opta-MatchHeader-Crested').find_all('td')[2].text.replace('\n        ', '').replace('\n      ', '')
    except AttributeError:
        team_one_score = 'None'
    
    team_one_score_lst.append(team_one_score)
    
    try:
        team_two = bs_data.find('table', class_='Opta-MatchHeader Opta-MatchHeader-Crested').find_all('td')[5].text.replace('\n        ', '').replace('\n      ', '')
    except AttributeError:
        team_two = 'None'
    
    team_two_lst.append(team_two)
    
    team_two_score = bs_data.find('table', class_='Opta-MatchHeader Opta-MatchHeader-Crested').find_all('td')[4].text.replace('\n        ', '').replace('\n      ', '')
    team_two_score_lst.append(team_two_score)
    
    # some of the games are yet to happen so not all information will be available which is why we try below
    
    try:
        referee = bs_data.find_all('div', 'Opta-Matchdata')[0].find('dd').text
    except IndexError:
        referee = 'None'
        
    referee_lst.append(referee)
    
    
    try:
        venue = bs_data.find_all('dd')[1].text
    except IndexError:
        venue = 'None'
        
    venue_lst.append(venue)
    
    try:
        discipline = bs_data.find_all('table', class_='Opta-Stats-Bars')[-1]
        penalties = discipline.find_all('tr')[1]
        team_one_penalties = penalties('td')[0].text
        team_two_penalties = penalties('td')[2].text

        reds = discipline.find_all('tr')[3]
        team_one_reds = reds('td')[0].text
        team_two_reds = reds('td')[2].text

        yellow = discipline.find_all('tr')[5]
        team_one_yellows = yellow('td')[0].text
        team_two_yellows = yellow('td')[2].text
    
    except IndexError:
        
        team_one_penalties = 'None'
        team_two_penalties = 'None'

        team_one_reds = 'None'
        team_two_reds = 'None'

        team_one_yellows = 'None'
        team_two_yellows = 'None'

    
    team_one_penalties_lst.append(team_one_penalties)
    team_two_penalties_lst.append(team_two_penalties)

    team_one_reds_lst.append(team_one_reds)
    team_two_reds_lst.append(team_two_reds)
    
    team_one_yellows_lst.append(team_one_yellows)
    team_two_yellows_lst.append(team_two_yellows)
    
    driver.close()
    print('Done!')

In [14]:
for link in all_links:
    grab_link_stats(link)

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
venue_lst

In [ ]:
df = pd.DataFrame()

df['rounds'] = rounds_lst
df['team_one'] = team_one_lst
df['team_two'] = team_two_lst
df['team_one_score'] = team_one_score_lst
df['team_two_score'] = team_two_score_lst
df['referee'] = referee_lst
df['venue'] = venue_lst
df['team_one_penalties'] = team_one_penalties_lst
df['team_two_penalties'] = team_two_penalties_lst
df['team_one_reds'] = team_one_reds_lst
df['team_two_reds'] = team_two_reds_lst
df['team_one_yellows'] = team_one_yellows_lst
df['team_two_yellows'] = team_two_yellows_lst

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.to_csv('rugby_penalties_%s.csv' % season)